In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.stats as sm_stats
import scipy.stats as sc_stats

# Setup

In [2]:
# create a dataframe from the clean data output file
df = pd.read_csv('statistics_output.csv', index_col=0)
reg_df = df.filter(['SNP Vote 2014', 'Claimant Count 2014', 'Degree 2014', 'Low-Skilled Employment 2014', 'Population 2014', 'Community Belonging 2014', 'Crime 2014'])
reg_df.rename(columns={'SNP Vote 2014': 'SNP Vote', 'Claimant Count 2014': 'Claimant Count', 'Degree 2014': 'Degree', 'Low-Skilled Employment 2014': 'Low-Skilled Employment', 'Population 2014': 'Population', 'Community Belonging 2014': 'Community', 'Crime 2014': 'Crime'}, inplace=True)

In [3]:
# define a function to create a regression model (OLS)
def create_OLS_model(y, x):
  model = sm.OLS(y, x)
  return model.fit()

In [42]:
# create a regression model for each independent variable
snp_vote_model = create_OLS_model(df['Yes Vote 2014'], sm.add_constant(df['SNP Vote 2014']))
claimant_count_model = create_OLS_model(df['Yes Vote 2014'], sm.add_constant(df['Claimant Count 2014']))
degree_model = create_OLS_model(df['Yes Vote 2014'], sm.add_constant(df['Degree 2014']))
low_skilled_model = create_OLS_model(df['Yes Vote 2014'], sm.add_constant(df['Low-Skilled Employment 2014']))
population_model = create_OLS_model(df['Yes Vote 2014'], sm.add_constant(df['Population 2014']))
community_model = create_OLS_model(df['Yes Vote 2014'], sm.add_constant(df['Community Belonging 2014']))
crime_model = create_OLS_model(df['Yes Vote 2014'], sm.add_constant(df['Crime 2014']))

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


# Linear Relationship

In [43]:
def regression_plot(independent_variable, variable_name):
  sns.regplot(data=df, x=independent_variable, y='Yes Vote 2014')
  plt.title('Variable Scatter Plot and Regression Line')
  plt.savefig(f'regression_{variable_name}.png', dpi=300)
  plt.close()

# Normality of Residuals

In [7]:
def qq_plot(model, variable_name):
  sm.qqplot(data=model.resid, line='s')
  plt.title(f'Q-Q Plot of Residuals ({variable_name})')
  plt.savefig(f'qq_plot_{variable_name}.png', dpi=300)
  plt.close()

In [8]:
def residual_histogram(model, variable_name):
  sns.histplot(x=model.resid, stat="density", linewidth=0, kde=True)
  plt.title(f'Histogram of Residuals ({variable_name})')
  plt.xlabel('Residuals')
  
  # plot corresponding normal curve
  mu, std = sc_stats.norm.fit(model.resid)
  x = np.linspace(model.resid.min(), model.resid.max(), 100) # generate some x values
  p = sc_stats.norm.pdf(x, mu, std) # calculate the y values for the normal curve
  sns.lineplot(x=x, y=p, color="orange")
  plt.savefig(f'residual_histogram_{variable_name}.png', dpi=300)
  plt.close()

In [9]:
def box_plot(model, variable_name):
  sns.boxplot(x=model.resid, showmeans=True)
  plt.title(f'Box Plot of Residuals ({variable_name})')
  plt.xlabel('Residuals')
  plt.savefig(f'box_plot_{variable_name}.png', dpi=300)
  plt.close()

In [10]:
def shapiro_wilk_test(model):
  return sc_stats.shapiro(model.resid)

# Independence of Residuals

In [11]:
def durbin_watson_test(model):
  return sm_stats.stattools.durbin_watson(model.resid)

# Homoscedasticity of Residuals

In [44]:
def residuals_fitted_plot(model, variable_name):
  sns.residplot(x=model.fittedvalues, y='Yes Vote 2014', data=df, lowess=True, scatter_kws={'alpha': 0.5}, line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
  plt.title(f'Residuals vs Fitted Values ({variable_name})')
  plt.xlabel('Fitted Values')
  plt.ylabel('Residuals')
  plt.savefig(f'residuals_fitted_plot_{variable_name}.png', dpi=300)
  plt.close()

# Multicollinearity

In [34]:
def correlation_matrix():
  plt.figure(figsize=(6, 5))
  sns.heatmap(reg_df.corr(), annot=True, fmt=".2f", cmap='vlag')
  plt.title('Correlation Matrix')
  plt.tight_layout()
  plt.savefig(f'correlation_matrix.png', dpi=300)
  plt.close()

In [41]:
def vif_analysis(df):
  # create a dataframe with the independent variables and a constant
  df_with_constant = sm.add_constant(reg_df)
  # create the VIF dataframe
  vif_df = pd.DataFrame()
  vif_df['feature'] = df_with_constant.columns
  # calculate VIF for each feature and add it to the VIF dataframe
  vif_df['VIF'] = [sm_stats.outliers_influence.variance_inflation_factor(df_with_constant.values, i) for i in range(len(df_with_constant.columns))]
  vif_df.drop([0], inplace=True)
  # return the VIF dataframe
  return vif_df

# Outliers

In [37]:
def test_cooks_distance(model, observations):
  influnetial_data_points = []
  counter = 0
  for cooks_distance in model.get_influence().cooks_distance[0]:
    if cooks_distance > 4/observations:
      influnetial_data_points.append(df.index[counter])
    counter += 1
  return influnetial_data_points

# Output

In [38]:
def assumptions_testing(model, independent_variable, variable_name, df=df, observations=32):
  sw_test = shapiro_wilk_test(model)
  dw_test = durbin_watson_test(model)
  influential_data_points = test_cooks_distance(model, observations)
  
  regression_plot(independent_variable, variable_name)
  qq_plot(model, variable_name)
  residual_histogram(model, variable_name)
  box_plot(model, variable_name)
  residuals_fitted_plot(model, variable_name)
  correlation_matrix()

  print(f'================== Regression Assumption Statistics ({variable_name}) ==================')
  print('\n')
  print(model.summary())
  print('\n')
  print(f'Shapiro-Wilk Test: {sw_test[0]:.3f} (p-value: {sw_test[1]:.3f})')
  print(f'Durbin Watson Test: {dw_test:.3f}')
  if len(influential_data_points) > 0:
    print(f'Number Of Influential Data Points Found: {len(influential_data_points)}')
    print(f'Influential Data Points Found: {influential_data_points}')
  else:
    print('No Influential Data Points Found')
  print('\n')
  print(f'========================== Regression Assumption Statistics ==========================')
  print('\n')
  print(vif_analysis(df))
  print('\n')

In [53]:
assumptions_testing(crime_model, 'Crime 2014', 'Crime')

================== Regression Assumption Statistics (Crime) ==================


                            OLS Regression Results                            
Dep. Variable:          Yes Vote 2014   R-squared:                       0.393
Model:                            OLS   Adj. R-squared:                  0.373
Method:                 Least Squares   F-statistic:                     19.44
Date:                Fri, 30 Dec 2022   Prob (F-statistic):           0.000123
Time:                        16:59:39   Log-Likelihood:                 53.032
No. Observations:                  32   AIC:                            -102.1
Df Residuals:                      30   BIC:                            -99.13
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
